In [1]:
import numpy as np
from help_functions import *

In [2]:
class SOM:
    def __init__(self, map_width, map_height, input_dim, neighbourhood_function,
                 distance_k, learning_rate=0.5, sigma=None, decay_type='exponential', beta=0.999):
        self.map_width = map_width
        self.map_height = map_height
        self.input_dim = input_dim
        self.learning_rate = learning_rate
        self.distance_k = distance_k
        self.beta = beta
        self.time = 1
        
        if sigma is None:
            self.sigma = max(map_width, map_height) / 2.0
        else:
            self.sigma = sigma
            
        if self.distance_k == np.inf:
            self.calculate_distance_func = chebyshev_distance
        elif self.distance_k == 1:
            self.calculate_distance_func = manhattan_distance
        elif self.distance_k == 2:
            self.calculate_distance_func = euclidean_distance
        elif self.distance_k < 1:
            raise ValueError('Distance must have positive non-zero k value')
        else:
            self.calculate_distance_func = lambda a, b, axis: generic_distance(a, b, axis, self.distance_k)
            
        # TODO add other weight initialization options
        self.weights = np.random.rand(self.map_width, self.map_height, self.input_dim)
        
        if neighbourhood_function == 'gaussian':
            self.neighbourhood_func = gaussian_neighbourhood
        elif neighbourhood_function == 'rectangular':
            self.neighbourhood_func = rectangular_neighbourhood
        elif neighbourhood_function == 'triangular':
            self.neighbourhood_func = triangular_neighbourhood
        elif neighbourhood_function == 'cosine':
            self.neighbourhood_func = cosine_down_to_zero_neighbourhood
        else:
            raise ValueError(f'Unknown neighbourhood function {neighbourhood_function}')
        
        
        if decay_type == 'exponential' and 0 < self.beta < 1:
            self.calculate_decay = decay_exponential
        elif decay_type == 'power' and self.beta < 0:
            self.calculate_decay = decay_power
        else:
            raise ValueError(f'Unknown decay type or invalid beta')
            
    def get_weights(self):
        return self.weights
    
    def get_weight_of_node(self, node_idx):
        return self.weights[node_idx[0]][node_idx[1]]
    
    def update_time(self):
        self.time += 1
        
    def find_BMU(self, input_vector):
        dists = self.calculate_distance_func(self.weights, input_vector, 2)

        min_index = np.argmin(dists)
        bmu_idx = np.unravel_index(min_index, dists.shape)
        return bmu_idx
    
    def calculate_grid_distances(self, bmu_idx):
        x_coords, y_coords = np.meshgrid(np.arange(self.map_width), 
                                         np.arange(self.map_height), indexing='ij')
        dist_sq = (x_coords - bmu_idx[0])**2 + (y_coords - bmu_idx[1])**2
        return np.sqrt(dist_sq)
            
    def calculate_neighbourhood_influence(self, bmu_idx, sigma_t):
        grid_dists = self.calculate_grid_distances(bmu_idx)
        return self.neighbourhood_func(grid_dists, sigma_t)
    
    def update_weights(self, input_vector, bmu_idx):
        eta_t = self.calculate_decay(self.learning_rate, self.beta, self.time)
        sigma_t = self.calculate_decay(self.sigma, self.beta, self.time)
        
        # shape (map_width, map_height)
        influence = self.calculate_neighbourhood_influence(bmu_idx, sigma_t)
        
        # shape (map_width, map_height, input_dim)
        diff = input_vector - self.weights
        
        # reshaping to (map_width, map_height, 1) to broadcast over diff
        influence_new = influence[:, :, np.newaxis]
        
        self.weights += eta_t * influence_new * diff
        
    def calculate_TE(self, data):
        diff_total = 0
        for sample in data:
            bmu_idx = self.find_BMU(sample)      
            weight = self.get_weight_of_node(bmu_idx)
            
            diff_total += self.calculate_distance_func(weight, sample, 0)
        
        return diff_total / data.shape[0]
            
    
    def train_online(self, data, num_epochs):
        num_samples = data.shape[0]
        
        for epoch in range(num_epochs):
            for sample in range(num_samples):
                input_vector = data[sample]
                
                bmu_idx = self.find_BMU(input_vector)
                
                self.update_weights(input_vector, bmu_idx)
                
            self.update_time()
            
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{num_epochs} complete. Current sigma: {self.calculate_decay(self.sigma, self.beta, self.time):.4f}, learning rate: {self.calculate_decay(self.learning_rate, self.beta, self.time):.4f}, TE: {self.calculate_TE(data):.4f}")
    
    

In [8]:
som = SOM(3,3,3,"gaussian",2)
som.get_weights()

array([[[0.5554615 , 0.0057432 , 0.31223437],
        [0.58027971, 0.57377901, 0.67795871],
        [0.93084516, 0.35614254, 0.06549178]],

       [[0.89387697, 0.03062472, 0.68075756],
        [0.61466186, 0.24685801, 0.35472175],
        [0.05993168, 0.73577321, 0.96932804]],

       [[0.86940729, 0.52119595, 0.33511375],
        [0.0478001 , 0.07198124, 0.93063403],
        [0.48881532, 0.48723484, 0.54312821]]])

In [9]:
data = np.array([[1, 1,1], [3, 4,3], [1, 8,4], [7, 8,7]])
som.train_online(data, 1000)

Epoch 10/1000 complete. Current sigma: 1.4836, learning rate: 0.4945, TE: 2.5510
Epoch 20/1000 complete. Current sigma: 1.4688, learning rate: 0.4896, TE: 2.5530
Epoch 30/1000 complete. Current sigma: 1.4542, learning rate: 0.4847, TE: 2.5554
Epoch 40/1000 complete. Current sigma: 1.4397, learning rate: 0.4799, TE: 2.5580
Epoch 50/1000 complete. Current sigma: 1.4254, learning rate: 0.4751, TE: 2.5458
Epoch 60/1000 complete. Current sigma: 1.4112, learning rate: 0.4704, TE: 2.5219
Epoch 70/1000 complete. Current sigma: 1.3971, learning rate: 0.4657, TE: 2.4966
Epoch 80/1000 complete. Current sigma: 1.3832, learning rate: 0.4611, TE: 2.4714
Epoch 90/1000 complete. Current sigma: 1.3695, learning rate: 0.4565, TE: 2.4466
Epoch 100/1000 complete. Current sigma: 1.3558, learning rate: 0.4519, TE: 2.4225
Epoch 110/1000 complete. Current sigma: 1.3423, learning rate: 0.4474, TE: 2.3989
Epoch 120/1000 complete. Current sigma: 1.3290, learning rate: 0.4430, TE: 1.8433
Epoch 130/1000 complete. 

In [10]:
som.get_weights()

array([[[6.98794776, 7.99517167, 6.99156061],
        [4.05413499, 7.99138825, 5.52315306],
        [1.00886643, 7.99156518, 4.00081778]],

       [[5.03185288, 6.03595247, 5.03429411],
        [3.01739076, 5.27708232, 3.77061473],
        [1.00670208, 4.56946794, 2.53289053]],

       [[3.00349627, 4.0023055 , 3.00350286],
        [2.02246959, 2.53793291, 2.02500677],
        [1.00296665, 1.01478557, 1.00739808]]])